## Введение и особенности LightGBM

__`LightGBM`__ - это фреймворк с реализацией градиетного бустинга, построенного на древовидных алгоритмах обучения. Основные приемущества данного фреймворка:

* быстрая скорость обучения и более высокая эффективность
* более низкая потребность в памяти
* лучшее качество
* поддержка параллельного обучения и обучения на GPU
* способность работы с крупномасштабными данными

## Оптимизация скорости и использования памяти

Многие реализации градиентного бустинга используют алгоритмы предварительной сортировки значений признаков (например, так работает `XGBoost`) для построения разбиения дерева решений. Это простое решение, но его нелегко оптимизировать.

`LightGBM` использует __алгоритмы на основе гистограмм__, которые объединяют непрерывные значения признаков (атрибутов) в отдельные интервалы (бины) гистограммы. Это значительно ускоряет обучение и снижает использование памяти. К преимуществам алгоритмов на основе гистограмм можно отнести следующее:

1. снижена стоимость расчета gain'а для каждого сплита
 * алгоритмы на основе предварительной сортировки имеют временную сложность __`O(#data)`__
 * алгоритм на основе гистограммы имеет временную сложность __`O(#bins)`__, при этом __#bins__ значительно меньше __#data__


2. использование разности гистограмм для дальнейшего ускорения процесс построения дерева
 * чтобы получить гистограммы одного листа в бинарном дереве, используется вычитание гистограмма родительской вершины и одной из производных вершин
 * строится гистограмма только для того листа, который имеет меньше __#data__, после чего можно получить гистограмму соседней вершины, путем вычитания гистограммы, это несложная операция, ее сложность __`O(#bins)`__.


3. Непрерывные значения заменяеются дискретными бинами. Если __#bins__ мало, то можно использовать тип данных, который занимает мало место в памяти, например, `uint8` для хранения обучающих данных. Кроме того, неот необходимости хранить дополнительную информацию с предварительно отсортированными значениями признаков.

## Оптимизация качества работы алгоритма

Большая часть алгоритмов, основанных на решающих деревьях, основаны на __Level-wise__ архитектуре деревьев, дерево строится по уровням, как показано на рисунке (на каждой итерации, производятся 2 дочерние вершины):

<img src="images/lightgbm_trees_1.png" width=500 height=400 />

__`LightGBM`__ строить деревья по листьям. Он выбирает для роста лист с максимальным приростом функции потерь. Удерживая фиксированное значение __#leaf__, алгоритмы построения деревьев на основе __Leaf-wise__ архитектуры, как правило, приводит к меньшему значению функции потерь, чем алгоритмы, основанные на __Level-wise__ архитектуре.

<img src="images/lightgbm_trees_2.png" width=500 height=400 />

## Оптимальное разбиения для категориальных признаков

Обычно, категориальные признаки обрабатывают с помощью `One-Hot-Encoding`, но этот подход неоптимальный для решающих деревьев. Например, для категориальных признаков с большим количеством уникальных значений, дерево, построенное на основе `One-Hot-Encoding` признаков, имеет тенденцию быть несбалансированным и должно быть очень глубоким для достижения хорошей точности.

Вместо `One-Hot-Encoding`, более оптимальным подходом к разбиению вершины по категориальному признаку, является разделение категориального признака на 2 подмножества. Если признак имеет `k` уникальных значений, то существует $2^{(k-1)} - 1$  возможных разбиений. В __`LightGBM`__ используется подход, который требует __O(klog(k))__ операций для поиска оптимального разбиения. Его основная идея состоит в том, чтобы отсортировать категории в соответствии со значением функции потерь для при каждом возможном разбиении. __`LightGBM`__ сортирует гистограмму категориального признака в соответствии с накопленными значениями `sum_gradient / sum_hessian`, и находит оптимальное разделение на отсортированной гистограмме.

## GOSS - случайный отбор на основе градиентов

__GOSS__ - Gradient Based One Side Sampling

Наблюдения с маленькими градиентами - наблюдения, на которых алгоритм работает хорошо, а наблюдения с большими градиентами - наблюдения, где алгоритм работает неуверенно и серьезно ошибается. Существует наивный подход к даунсэмплингу, суть метода в том, чтобы отбросить наблюдения с небольшими градиентами, оставив только примеры с большими градиентами. __НО!__ Такой подход изменит распределение данных. 

Несмотря на то, что в градиетном бустинге наблюдения не имеют весов, можно заметить, что наблюдения с разными градиентами вносят различный вклад в процесс вычисления информационного выигрыша: объекты с большим градиетном вносят больший вклад в информационный выигрыш. Поэтому, чтобы сохранить точность оценки онформационного выигрыша, необходимо сохранять наблюдения с большими градиентами, то есть брать наблюдения с градиентами выше заранее заданного порога или ориентироваться на верхние процентили, и случайным образом удалять наблюдения с маленьким градиентом. Такой подходит приводит к более точной оценке выигрыша по сравнению с равномерной случайной выборкой.

__Алгоритм GOSS__:
* отсортировать наблюдения по значению градиента по убыванию;
* выбрать верхние экземпляры `a * 100%` с наибольшим значением градиента;
* произвольно выбрать `b * 100%` наблюдения из оставшихся данных;
* без пунтка 3, количество объектов, имеющих небольшой градиент, было бы равно `1 - a`. Для сохранения исходного распределения, `LightGBM` усиливает вклад наблюдений, имеющих небольшие градиенты, на константу, равную `(1 - a) / b`, чтобы придать большие вес наблюдениям с большим градиентом, но не сильно изменить исходное распределение в данных.

## EFB - связывание взаимоисключающих признаков

__EFB__ - Exclusive Feature Bundling

Давай вспомним, что для построения гистограммы требуется __O(#data)__, причем __`#data = #rows * #cols`__. Если мы сможем уменьшить выборку __#cols__ (то же самое, что и __#features__), мы значительно ускорим процесс построение дерева. В __`LightGBM`__ это достигается за счет объединения признаков. Обычно, мы работаем с данными большой размерности, и, как правило, признаковое пространство является довольно сильно разреженным, что дает нам возможность скоратить количество признаков без больших потерь в качестве. В разреженном пространстве, многие признаки уникальны, то есть редко принимают ненулевые значения одновременно. В качестве примера можно рассмотреть `One-Hot-Encoding` признаки, такие взаимоисключающие признаки можно связывать без потери качества. __`LightGBM`__ безопасно идентифицирует такие признаки и объединяет их в один признак, таким образом сложность построения гистограммы снижается до __O(#rows * #bundles)__, где `#bundles << #features`.

__Алгоритм EFB__:
* построить граф с взвешенными ребрами; вес ребра определяется как доля исключительных признаков, которые имеют непересекающиеся ненулевые значения;

* отсортировать признаки по количеству ненулевых примеров в порядке убывания;

* просмотреть упорядоченный список признаков и отнести признак к сгруппированному набору, если значение метрики не превышает порог, или создадим новый набор, если значение метрики превышает порог.

## Как параллелится градиетный бустинг? 
### мой любимый вопрос на собеседовании :)

### Традиционный подход
Традиционный подход направлен на параллелизацию процесса `поиска наилучшего разбиения` в дереве решений. __Процедура__ традиционного параллелизации выглядит следующим образом:

* разделить данные по вертикали (разные ядра / разные workers имеют разные наборы признаков)
* worker'ы находят локальную лучшую точку разделения на локальном подмножестве признаков
* собрать все лучшие локальные разбиения вместе и выбрать среди лучших локальных разбиений, самое лучшее
* остальные worker'ы разбивают данные согласно полученному оптимальному разбиению

__Недостатки__ традиционного подхода:

* имеет накладные расходы на вычисления, поскольку не может ускорить разбиение вершины, временная сложность разбиения равна __O(#data)__. Таким образом, прирост скорости не очень большой при большом __#data__.

### Feature Parallel

Поскольку традиционный `Feature Parallel` медленно работает при большом __#data__, внесем небольшое изменение: вместо вертикального разделения данных, каждый worker будет хранить полную обучающую выборку (все данные). Таким образом, __`LightGBM`__ не нуждается в обмене лучшими локальными разбиениями, поскольку каждый worker знает весь набор данных. Процедура `Feature Parallel` в __`LightGBM`__ выглядит следующим образом:

* worker'ы находят наилучшую точку разбиения (признак и значение признака) на локальном подмножестве признаков;
* сравнить все лучшие локальные разбиения и выбрать оптимальное;
* выполнить выбранное разбиение дерева;

Такое подход к параллелизации алгоритма, по-прежнему страдает от накладных расходов на вычисление для сплита, когда __#data__ все еще велик. Поэтому лучше использовать `Data Parallel`.

### Traditional Data Parallel

Традиционный алгоритм направлен на распапаллеливание всего процесса обучения модели:

* разбиваем данные по горизонтали;
* worker'ы используют локальные данные для построения локальных гистограмм;
* все локальные гистограммы объединяются в глобальную гистограмму;
* находим наилучшее разбиение в глобальной гистограмме и используем его для выращивания дерева решений;

В __`LightGBM`__, используется объединение не всех локальных гистограмм, а объединяются максимально непохожие друг на друга гистограммы, в идеале, объединяются только не пересекающиеся гистограммы. После чего worker'ы находят наилучшее локальное разбиение на объединенных локальных гистограммах и выбирается оптимальное разбиение (глобальное). Кроме того, выше, мы обсуждали, что мы можем получать гистограммы путем вычитания гистограммы потомка из гистограммы предка. Таким образом, мы еще сильнее можем ускорить `Data Parallel`.

In [1]:
import pandas as pd

## Пример EFB

Попробуем понять интуицию объединения признаков на примере. Перед эти еще раз очертим основную идею метода: EFB объединяет признаки, чтобы упростить обучение. Для того, что объединение признаков было обратимым, будем хранить уникальные признаки в разных бинах.

In [2]:
efb_data = pd.DataFrame({
    "feature_1": [0, 0, 0, 1, 2, 3, 4],
    "feature_2": [2, 1, 2, 0, 0, 0, 0],
    "feature_bundle": [6, 5, 6, 1, 2, 3, 4]
})
efb_data

,feature_1,feature_2,feature_bundle
0,0,2,6
1,0,1,5
2,0,2,6
3,1,0,1
4,2,0,2
5,3,0,3
6,4,0,4


В данном примере, видно, что признаки `feature_1` и  `feature_2` исключают друг друга. Чтобы добиться неперекрывающихся сегментов, добавим размер пакета для `feature_1` к `feature_2`. Это гарантирует, что ненулевые примеры объединенного признака `feature_1 & feature_2` находятся в разных сегментах. В признаке `feature_bundle` значения `1-4` содержат ненулевые примеры для `feature_1`, а значения `5-6` содержат ненулевые значения признака `feature_2`.

In [3]:
import numpy as np
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [5]:
data = pd.read_csv(
    "data/train.csv"
)
print("data.shape = {} rows, {} cols".format(*data.shape))

data.shape = 200000 rows, 202 cols


In [6]:
target = data["target"]
data = data.drop(["ID_code", "target"], axis=1)
print("data.shape = {} rows, {} cols".format(*data.shape))

data.shape = 200000 rows, 200 cols


In [7]:
x_train, x_valid = train_test_split(
    data, train_size=0.8, random_state=1
)
y_train, y_valid = train_test_split(
    target, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))

x_train.shape = 160000 rows, 200 cols
x_valid.shape = 40000 rows, 200 cols


## LightGBM API

In [8]:
params = {
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "auc",
    "learning_rate": 0.01,
    "n_estimators": 200,
    "n_jobs": 6,
    "seed": 27
}

In [16]:
dtrain = lgb.Dataset(
    data=x_train, label=y_train
)
dvalid = lgb.Dataset(
    data=x_valid, label=y_valid
)

model = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=200,
    valid_sets=[dtrain, dvalid],
    categorical_feature="auto",
    early_stopping_rounds=50,
    verbose_eval=10
)

Training until validation scores don't improve for 50 rounds
[10]	training's auc: 0.716713	valid_1's auc: 0.694117
[20]	training's auc: 0.729973	valid_1's auc: 0.707004
[30]	training's auc: 0.741827	valid_1's auc: 0.71714
[40]	training's auc: 0.753216	valid_1's auc: 0.72623
[50]	training's auc: 0.762935	valid_1's auc: 0.734123
[60]	training's auc: 0.774779	valid_1's auc: 0.7445
[70]	training's auc: 0.784419	valid_1's auc: 0.753455
[80]	training's auc: 0.793025	valid_1's auc: 0.761322
[90]	training's auc: 0.800332	valid_1's auc: 0.768126
[100]	training's auc: 0.80644	valid_1's auc: 0.77331
[110]	training's auc: 0.812058	valid_1's auc: 0.777985
[120]	training's auc: 0.81761	valid_1's auc: 0.78277
[130]	training's auc: 0.822393	valid_1's auc: 0.786508
[140]	training's auc: 0.826481	valid_1's auc: 0.790199
[150]	training's auc: 0.830668	valid_1's auc: 0.79387
[160]	training's auc: 0.834404	valid_1's auc: 0.7968
[170]	training's auc: 0.838368	valid_1's auc: 0.799991
[180]	training's auc: 0.

In [17]:
params["boosting_type"] = "goss"

model = lgb.train(
    params=params,
    train_set=dtrain,
    num_boost_round=200,
    valid_sets=[dtrain, dvalid],
    categorical_feature="auto",
    early_stopping_rounds=50,
    verbose_eval=10
)

Training until validation scores don't improve for 50 rounds
[10]	training's auc: 0.716713	valid_1's auc: 0.694117
[20]	training's auc: 0.729973	valid_1's auc: 0.707004
[30]	training's auc: 0.741827	valid_1's auc: 0.71714
[40]	training's auc: 0.753216	valid_1's auc: 0.72623
[50]	training's auc: 0.762935	valid_1's auc: 0.734123
[60]	training's auc: 0.774779	valid_1's auc: 0.7445
[70]	training's auc: 0.784419	valid_1's auc: 0.753455
[80]	training's auc: 0.793025	valid_1's auc: 0.761322
[90]	training's auc: 0.800332	valid_1's auc: 0.768126
[100]	training's auc: 0.80644	valid_1's auc: 0.77331
[110]	training's auc: 0.813384	valid_1's auc: 0.779508
[120]	training's auc: 0.819688	valid_1's auc: 0.785428
[130]	training's auc: 0.825127	valid_1's auc: 0.790613
[140]	training's auc: 0.830092	valid_1's auc: 0.795163
[150]	training's auc: 0.834767	valid_1's auc: 0.799409
[160]	training's auc: 0.838595	valid_1's auc: 0.802016
[170]	training's auc: 0.842105	valid_1's auc: 0.804969
[180]	training's au

## LightGBM Cross-Validation

In [21]:
cv_result = lgb.cv(
    params=params,
    train_set=dtrain,
    num_boost_round=200,
    categorical_feature="auto",
    early_stopping_rounds=50,
    verbose_eval=10,
    stratified=True,
    shuffle=True,
    nfold=5, 
)

/Users/a17582007/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:502: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[10]	cv_agg's auc: 0.69654 + 0.00325503
[20]	cv_agg's auc: 0.711838 + 0.00368813
[30]	cv_agg's auc: 0.722289 + 0.00442268
[40]	cv_agg's auc: 0.731624 + 0.00481065
[50]	cv_agg's auc: 0.740605 + 0.00457461
[60]	cv_agg's auc: 0.750053 + 0.00390583
[70]	cv_agg's auc: 0.758772 + 0.00325518
[80]	cv_agg's auc: 0.765526 + 0.00264697
[90]	cv_agg's auc: 0.77133 + 0.00212234
[100]	cv_agg's auc: 0.776675 + 0.00173496
[110]	cv_agg's auc: 0.782948 + 0.00138662
[120]	cv_agg's auc: 0.788478 + 0.00168213
[130]	cv_agg's auc: 0.793202 + 0.00138722
[140]	cv_agg's auc: 0.797387 + 0.00151427
[150]	cv_agg's auc: 0.801 + 0.00142765
[160]	cv_agg's auc: 0.804358 + 0.00156804
[170]	cv_agg's auc: 0.807733 + 0.00162715
[180]	cv_agg's auc: 0.810735 + 0.00189044
[190]	cv_agg's auc: 0.813285 + 0.00188992
[200]	cv_agg's auc: 0.815868 + 0.00198339


## LightGBM Sklearn-API

In [23]:
model = lgb.LGBMClassifier(**params)
model.fit(
    X=x_train,
    y=y_train,
    eval_set=[(x_train, y_train), (x_valid, y_valid)],
    early_stopping_rounds=25,
    eval_metric="auc",
    verbose=10
)

Training until validation scores don't improve for 25 rounds
[10]	training's auc: 0.716713	valid_1's auc: 0.694117
[20]	training's auc: 0.729973	valid_1's auc: 0.707004
[30]	training's auc: 0.741827	valid_1's auc: 0.71714
[40]	training's auc: 0.753216	valid_1's auc: 0.72623
[50]	training's auc: 0.762935	valid_1's auc: 0.734123
[60]	training's auc: 0.774779	valid_1's auc: 0.7445
[70]	training's auc: 0.784419	valid_1's auc: 0.753455
[80]	training's auc: 0.793025	valid_1's auc: 0.761322
[90]	training's auc: 0.800332	valid_1's auc: 0.768126
[100]	training's auc: 0.80644	valid_1's auc: 0.77331
[110]	training's auc: 0.813384	valid_1's auc: 0.779508
[120]	training's auc: 0.819688	valid_1's auc: 0.785428
[130]	training's auc: 0.825127	valid_1's auc: 0.790613
[140]	training's auc: 0.830092	valid_1's auc: 0.795163
[150]	training's auc: 0.834767	valid_1's auc: 0.799409
[160]	training's auc: 0.838595	valid_1's auc: 0.802016
[170]	training's auc: 0.842105	valid_1's auc: 0.804969
[180]	training's au

LGBMClassifier(boosting_type='goss', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.01, max_depth=-1,
               metric='auc', min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=200, n_jobs=6, num_leaves=31,
               objective='binary', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, seed=27, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)